In [1]:
import os
import config

import pandas as pd
import numpy as np
from scipy.stats import beta

from flask import Flask, jsonify, render_template

import mysql.connector

from datetime import datetime, date, time, timedelta
from time import sleep

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
import requests

import firebase_admin
from firebase_admin import credentials
from firebase_admin import db


In [2]:
# Fetch the service account key JSON file contents
cred = credentials.Certificate(config.firebase_file_path)
# Initialize the app with a service account, granting admin privileges
firebase_admin.initialize_app(cred, {
    'databaseURL': config.databaseURL
})



In [3]:
listing_stats_db = "listing_stats.json"
r = requests.get(config.databaseURL + listing_stats_db)
r = r.json()

if r:
    data = [r[i] for i in r]
    listingstats_df = pd.DataFrame.from_dict(data, orient='columns')

listingstats_df.head()

,listingID,record_date,revenue,sold,visits
0,706002567,7/9/19,0,0,1
1,704569159,7/9/19,0,0,0
2,703387331,7/9/19,0,0,0
3,702303569,7/9/19,0,0,1
4,701344590,7/9/19,0,0,0


In [4]:
listingstats_df['record_date']= pd.to_datetime(listingstats_df['record_date'])

In [5]:
listingstats_df["record_date_local"] = listingstats_df.record_date.dt.tz_localize('US/Central')

In [6]:
listingstats_df['week'] = (listingstats_df['record_date_local'] + pd.DateOffset(days=1)).dt.week

listingstats_df=listingstats_df.sort_values(by='record_date',ascending=False)

In [7]:
listingstats_df = listingstats_df.astype({"visits": int, "sold": int, 'revenue':float})


In [8]:
listingstats_df.sample(10)

,listingID,record_date,revenue,sold,visits,record_date_local,week
1446,680082346,2019-05-12,0.0,0,1,2019-05-12 00:00:00-05:00,20
178,701341356,2019-07-03,0.0,0,2,2019-07-03 00:00:00-05:00,27
2079,680082346,2019-03-29,0.0,0,0,2019-03-29 00:00:00-05:00,13
371,680071910,2019-06-27,0.0,0,0,2019-06-27 00:00:00-05:00,26
1843,680064082,2019-04-18,0.0,0,16,2019-04-18 00:00:00-05:00,16
1107,693947901,2019-05-27,0.0,0,0,2019-05-27 00:00:00-05:00,22
1985,680077296,2019-04-06,0.0,0,0,2019-04-06 00:00:00-05:00,14
2822,685451744,2019-08-12,0.0,0,0,2019-08-12 00:00:00-05:00,33
2109,680064082,2019-03-27,0.0,0,0,2019-03-27 00:00:00-05:00,13
2455,721879701,2019-07-22,0.0,0,0,2019-07-22 00:00:00-05:00,30


In [9]:
cur_week_no=listingstats_df.iloc[0]['week']

cur_week_stats=listingstats_df.loc[listingstats_df['week'] == cur_week_no,:]

cur_week_stats.head()
cur_week_stats.dtypes

listingID                                object
record_date                      datetime64[ns]
revenue                                 float64
sold                                      int64
visits                                    int64
record_date_local    datetime64[ns, US/Central]
week                                      int64
dtype: object

In [10]:
cur_week_stats.dtypes

listingID                                object
record_date                      datetime64[ns]
revenue                                 float64
sold                                      int64
visits                                    int64
record_date_local    datetime64[ns, US/Central]
week                                      int64
dtype: object

In [11]:
visits=[]

cur_week_stats = cur_week_stats.astype({"visits": int, "sold": int, 'revenue':int})
cur_week_stats.dtypes

listingID                                object
record_date                      datetime64[ns]
revenue                                   int64
sold                                      int64
visits                                    int64
record_date_local    datetime64[ns, US/Central]
week                                      int64
dtype: object

In [12]:
cur_visits_perweek=cur_week_stats['visits'].sum()
cur_visits_perweek

48

In [13]:

prev_week_no=cur_week_no-1
prev_week_stats=listingstats_df.loc[listingstats_df['week'] == prev_week_no,:]
prev_week_stats.head()

,listingID,record_date,revenue,sold,visits,record_date_local,week
2960,687898920,2019-08-17,0.0,0,2,2019-08-17 00:00:00-05:00,33
2967,693930997,2019-08-17,0.0,0,0,2019-08-17 00:00:00-05:00,33
2972,721879701,2019-08-17,0.0,0,0,2019-08-17 00:00:00-05:00,33
2971,721881161,2019-08-17,0.0,0,0,2019-08-17 00:00:00-05:00,33
2970,701339360,2019-08-17,0.0,0,3,2019-08-17 00:00:00-05:00,33


In [14]:
prev_visits_perweek=prev_week_stats['visits'].sum()

visits_diff=(cur_visits_perweek/prev_visits_perweek)*100
visits_diff = str(round(visits_diff, 1))

# visits_diff='{0:+d}'.format(visits_diff)

visits.append(str(cur_visits_perweek))
visits.append(visits_diff+'%')

visits

['48', '33.3%']

In [15]:
favs_db = "favorites.json"
r = requests.get(config.databaseURL + favs_db)
r = r.json()

if r:
    data = [r[i] for i in r]
    favs_df = pd.DataFrame.from_dict(data, orient='columns')
    
favs_df.head()

,buyerID,dateLiked,listingID
0,tinareyes1106,7/24/19,687898920
1,laurensexton05,7/24/19,687898920
2,savnrgy,7/23/19,701344590
3,vtorreschi2,7/23/19,680064082
4,valeriegilroy1,7/23/19,693930997


In [16]:
favs_df['dateLiked']= pd.to_datetime(favs_df['dateLiked'])

In [17]:
favs_df["dateLiked_local"] = favs_df.dateLiked.dt.tz_localize('US/Central')

In [18]:
favs_df['week'] = (favs_df['dateLiked_local'] + pd.DateOffset(days=1)).dt.week
cur_week_favs=favs_df.loc[favs_df['week'] == cur_week_no,:]

cur_wk_fav_count=cur_week_favs['buyerID'].count()

prev_week_favs=favs_df.loc[favs_df['week'] == prev_week_no,:]
prev_wk_fav_count=prev_week_favs['buyerID'].count()

favs=[]

wk_fav_diff=(cur_wk_fav_count/prev_wk_fav_count)*100
wk_fav_diff = str(round(wk_fav_diff, 1))

favs.append(str(cur_wk_fav_count))
favs.append(wk_fav_diff+'%')

listing_stats_df=pd.DataFrame({'visitsPerWeek':visits,
                             'favsPerWeek':favs})

list_stats=[]

for index, item in listing_stats_df.iterrows():
    stats={}
    stats['avisitsPerWeek']=item[0]
    stats['favsPerWeek']=(item[1])

    list_stats.append(stats)

list_stats

[{'avisitsPerWeek': '48', 'favsPerWeek': '10'},
 {'avisitsPerWeek': '33.3%', 'favsPerWeek': '25.0%'}]